In [1]:
import numpy as np
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.models import Sequential
import pandas as pd


In [25]:
with open ("dostoevsky.txt", encoding='utf-8') as _in:
  lines = []
  for line in _in:
    line = line.strip().lower()
    line = line.replace('\xa0-','')
    line = line.replace('\xa0','')
    if len(line) == 0:
      continue
    lines.append(line)

text = ''.join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

seqlen, step = 10, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - seqlen, step):
  input_chars.append(text[i: i + seqlen])
  label_chars.append(text[i + seqlen])

X = np.zeros((len(input_chars), seqlen, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
  for j, ch in enumerate(input_char):
    X[i, j, char2index[ch]] = 1
  y[i, char2index[label_chars[i]]] = 1

<ipython-input-25-d7b2c79fd6ec>:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_chars), seqlen, nb_chars), dtype=np.bool)
<ipython-input-25-d7b2c79fd6ec>:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)


In [28]:
batch_size = [64, 128, 256]
hidden_size = [32, 128, 512]
num_iterations = 4
num_epochs_per_interation = 1
num_preds_per_epoch = 15

results = []
row_data = []

for batch in batch_size:

  for hidden in hidden_size:

    model = Sequential()
    model.add(LSTM(hidden, input_shape=(seqlen, nb_chars)))
    model.add(Dense(nb_chars))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    fit_data = model.fit(X, y, batch_size=batch, epochs=num_epochs_per_interation)

    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    print(f"Генерация из посева: {test_chars}")
    print(test_chars, end="")

    for i in range(num_preds_per_epoch):

      X_test = np.zeros((1, seqlen, nb_chars))
      for j , ch in enumerate(test_chars):
        X_test[0, j, char2index[ch]] = 1

      pred = model.predict(X_test, verbose=0)[0]
      y_pred = index2char[np.argmax(pred)]

      print(y_pred, end="")

      test_chars = test_chars[1:] + y_pred

    loss_array = np.array(fit_data.history['loss'])
    data_append = {'batch_size': str(batch),  'number_of_neurons': str(hidden), 'loss': loss_array}
    row_data.append(data_append)

experiment_data = pd.DataFrame(row_data)


3012/3012 [==============================] - 34s 11ms/step - loss: 2.8593
Генерация из посева: тала с мес
3012/3012 [==============================] - 92s 30ms/step - loss: 2.5819
Генерация из посева: ою собстве
3012/3012 [==============================] - 954s 315ms/step - loss: 2.4197
Генерация из посева: -то не при
1506/1506 [==============================] - 21s 12ms/step - loss: 2.9812
Генерация из посева: тянул к то
1506/1506 [==============================] - 57s 36ms/step - loss: 2.7259
Генерация из посева: уже изноше
1506/1506 [==============================] - 491s 324ms/step - loss: 2.5360
Генерация из посева: только вед
753/753 [==============================] - 17s 20ms/step - loss: 3.1361
Генерация из посева: от день. ч
753/753 [==============================] - 52s 65ms/step - loss: 2.8496
Генерация из посева:  если бы р
753/753 [==============================] - 354s 467ms/step - loss: 2.6663
Генерация из посева: для того, 
для того, и подел в по

In [29]:
experiment_data

,batch_size,number_of_neurons,loss
0,64,32,[2.859325647354126]
1,64,128,[2.5818989276885986]
2,64,512,[2.419746160507202]
3,128,32,[2.9811787605285645]
4,128,128,[2.7258598804473877]
5,128,512,[2.536012649536133]
6,256,32,[3.1361143589019775]
7,256,128,[2.8496344089508057]
8,256,512,[2.666344404220581]


Обучаем модель с оптимальными параметрами, полученными на предудущем шаге 
**batch_size** = 64
**neurons** = 512

In [30]:
model = Sequential()
model.add(LSTM(512, input_shape=(seqlen, nb_chars)))
model.add(Dense(nb_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

for iteration in range(7):
  print(f"  Итерация # {iteration}")
  fit_data = model.fit(X, y, batch_size=64, epochs=num_epochs_per_interation)

  test_idx = np.random.randint(len(input_chars))
  test_chars = input_chars[test_idx]

  print(f"Генерация из посева: {test_chars}")
  print(test_chars, end="")

  for i in range(num_preds_per_epoch):

    X_test = np.zeros((1, seqlen, nb_chars))
    for j , ch in enumerate(test_chars):
      X_test[0, j, char2index[ch]] = 1

    pred = model.predict(X_test, verbose=0)[0]
    y_pred = index2char[np.argmax(pred)]

    print(y_pred, end="")

    test_chars = test_chars[1:] + y_pred

  Итерация # 0
3012/3012 [==============================] - 890s 295ms/step - loss: 2.4317
Генерация из посева: н думал о 
н думал о не столо он   Итерация # 1
3012/3012 [==============================] - 839s 278ms/step - loss: 1.9811
Генерация из посева: третьем ча
третьем часто в перед   Итерация # 2
3012/3012 [==============================] - 827s 275ms/step - loss: 1.7628
Генерация из посева: просить, т
просить, то не вот ста  Итерация # 3
3012/3012 [==============================] - 829s 275ms/step - loss: 1.6050
Генерация из посева: ю; но даже
ю; но даже в самом дел  Итерация # 4
3012/3012 [==============================] - 793s 263ms/step - loss: 1.4699
Генерация из посева: ил дверь и
ил дверь и потому что   Итерация # 5
3012/3012 [==============================] - 780s 259ms/step - loss: 1.3336
Генерация из посева: ка не спал
ка не спал он с ней и   Итерация # 6
3012/3012 [==============================] - 782s 259ms/step - loss: 1.1940
Генерация из посева: уединился 
уединил

*можно было продолжать, но слишком все медленно происходит, ждать совсем нет возможности ((

#### Выводы:
1. На результат влияет количество итераций - чем больше итераций, тем лучше результат
2. Положительно влияет количество нейронов в скрытом слое, чем больше нейронов тем лучше результат